In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,saw,saw
band,[9],7
commitnumber,1013f91,5014a19
conc,-,-
dv,-,0.001
klin,-,0
molecule,{'h2o': 'atmpro'},h2o
ng_adju,-,[0]
ng_refs,-,[7]
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.001259                    1  0.000295
0.000750        2 -0.001142                    2  0.000246
0.001052        3 -0.000861                    3  0.000605
0.001476        4 -0.000511                    4  0.000834
0.002070        5 -0.000086                    5  0.001099
0.002904        6  0.000426                    6  0.001402
0.004074        7  0.001038                    7  0.001757
0.005714        8  0.001761                    8  0.002176
0.008015        9  0.002612                    9  0.002656
0.011243       10  0.003607                   10  0.003206
0.015771       11  0.004761                   11  0.003835
0.022122       12  0.006094                   12  0.004553
0.031031       13  0.007617                   13  0.005371
0.043528       14  0.009131                   14  0.006184
0.061057       15  0.010389                   15  0.006875
0.085645       16  0.011526                   16  0.007523
0.120136       17  0.012713                   17  0.008216
0.168516       18  0.013951                   18  0.008957
0.236378       19  0.015242                   19  0.009747
0.331549       20  0.016599                   20  0.010597
0.465100       21  0.017884                   21  0.011431
0.652400       22  0.016329                   22  0.010827
0.915100       23  0.011701                   23  0.008511
1.283650       24  0.007515                   24  0.006220
1.800600       25  0.004612                   25  0.004454
2.525700       26  0.002662                   26  0.003109
3.542800       27  0.001392                   27  0.002098
4.969550       28  0.000710                   28  0.001433
6.970850       29  0.000442                   29  0.001054
9.778100       30  0.000315                   30  0.000800
13.715850      31  0.000212                   31  0.000591
19.239350      32  0.000125                   32  0.000427
26.987250      33  0.000127                   33  0.000367
37.855300      34  0.000218                   34  0.000406
53.100050      35  0.000308                   35  0.000465
73.887500      36  0.000385                   36  0.000517
97.662500      37  0.000437                   37  0.000539
121.437500     38  0.000435                   38  0.000510
145.212500     39  0.000533                   39  0.000598
168.987500     40  0.000586                   40  0.000617
192.762500     41  0.000588                   41  0.000564
216.537500     42  0.000490                   42  0.000406
240.312500     43  0.000320                   43  0.000193
264.087500     44  0.000055                   44 -0.000093
287.862500     45 -0.000336                   45 -0.000470
311.637500     46 -0.000471                   46 -0.000566
335.412500     47  0.000097                   47  0.000031
359.187500     48  0.001223                   48  0.001156
382.962500     49  0.003170                   49  0.003026
406.737500     50  0.006179                   50  0.005957
430.512500     51  0.009862                   51  0.009629
454.287500     52  0.014103                   52  0.013583
478.062500     53  0.018802                   53  0.017775
501.837500     54  0.023892                   54  0.022592
525.612500     55  0.029291                   55  0.028272
549.387500     56  0.034865                   56  0.034238
573.162500     57  0.040476                   57  0.039780
596.937500     58  0.045917                   58  0.044668
620.712500     59  0.050988                   59  0.048981
644.487500     60  0.055502                   60  0.053353
668.262500     61  0.059334                   61  0.057482
692.037500     62  0.062449                   62  0.061331
715.812500     63  0.064898                   63  0.064654
739.587500     64  0.066843                   64  0.067336
763.362500     65  0.068540                   65  0.069461
78

# Fluxes by Layer

CRD                            CLIRAD                \
                       flug          fldg     fnetg      flug          fldg   
pressure    level                                                             
0.000000    1     -8.627833  0.000000e+00 -8.627833 -8.686979  9.397626e-08   
0.000624    2     -8.627833  9.079452e-08 -8.627833 -8.686979  1.436867e-07   
0.000876    3     -8.627833  1.303413e-07 -8.627833 -8.686979  1.703131e-07   
0.001229    4     -8.627833  1.952246e-07 -8.627833 -8.686979  2.164066e-07   
0.001723    5     -8.627833  3.017399e-07 -8.627833 -8.686979  2.856332e-07   
0.002417    6     -8.627834  4.762605e-07 -8.627833 -8.686979  3.915969e-07   
0.003391    7     -8.627834  7.617428e-07 -8.627833 -8.686979  5.560210e-07   
0.004757    8     -8.627834  1.227917e-06 -8.627833 -8.686979  8.150606e-07   
0.006672    9     -8.627835  1.987182e-06 -8.627833 -8.686979  1.227644e-06   
0.009359    10    -8.627835  3.221389e-06 -8.627832 -8.686979  1.886905e-06   
0.013128    11    -8.627835  5.223075e-06 -8.627830 -8.686978  2.941907e-06   
0.018415    12    -8.627836  8.460646e-06 -8.627827 -8.686978  4.631354e-06   
0.025830    13    -8.627836  1.368225e-05 -8.627822 -8.686976  7.337541e-06   
0.036232    14    -8.627835  2.207239e-05 -8.627812 -8.686974  1.166850e-05   
0.050823    15    -8.627832  3.528800e-05 -8.627797 -8.686970  1.848535e-05   
0.071291    16    -8.627827  5.544345e-05 -8.627771 -8.686964  2.893160e-05   
0.100000    17    -8.627818  8.566377e-05 -8.627732 -8.686954  4.474940e-05   
0.140271    18    -8.627802  1.307462e-04 -8.627672 -8.686939  6.868201e-05   
0.196760    19    -8.627776  1.976102e-04 -8.627578 -8.686915  1.048593e-04   
0.275997    20    -8.627731  2.960771e-04 -8.627435 -8.686878  1.594843e-04   
0.387100    21    -8.627657  4.397814e-04 -8.627217 -8.686821  2.419016e-04   
0.543100    22    -8.627532  6.458637e-04 -8.626886 -8.686734  3.656011e-04   
0.761700    23    -8.627362  8.983573e-04 -8.626464 -8.686618  5.305715e-04   
1.068500    24    -8.627183  1.144871e-03 -8.626038 -8.686496  7.175129e-04   
1.498800    25    -8.627015  1.359903e-03 -8.625655 -8.686379  9.173638e-04   
2.102400    26    -8.626863  1.537780e-03 -8.625325 -8.686272  1.129443e-03   
2.949000    27    -8.626734  1.675964e-03 -8.625058 -8.686184  1.352975e-03   
4.136600    28    -8.626639  1.776424e-03 -8.624863 -8.686123  1.587299e-03   
5.802500    29    -8.626579  1.857100e-03 -8.624722 -8.686094  1.841596e-03   
8.139200    30    -8.626549  1.949214e-03 -8.624600 -8.686099  2.138077e-03   
11.417000   31    -8.626553  2.075708e-03 -8.624477 -8.686147  2.497073e-03   
16.014700   32    -8.626616  2.254162e-03 -8.624362 -8.686264  2.935599e-03   
22.464000   33    -8.626780  2.514395e-03 -8.624266 -8.686483  3.480725e-03   
31.510500   34    -8.627077  2.946976e-03 -8.624130 -8.686826  4.216741e-03   
44.200100   35    -8.627511  3.709424e-03 -8.623802 -8.687300  5.300380e-03   
62.000000   36    -8.628121  4.969041e-03 -8.623152 -8.687942  6.923977e-03   
85.775000   37    -8.628921  6.854286e-03 -8.622066 -8.688771  9.208940e-03   
109.550000  38    -8.629707  8.872782e-03 -8.620834 -8.689581  1.153783e-02   
133.325000  39    -8.630541  1.093279e-02 -8.619608 -8.690435  1.382910e-02   
157.100000  40    -8.631816  1.370774e-02 -8.618108 -8.691723  1.680073e-02   
180.875000  41    -8.633662  1.720440e-02 -8.616457 -8.693560  2.037772e-02   
204.650000  42    -8.636266  2.146564e-02 -8.614800 -8.696123  2.452927e-02   
228.425000  43    -8.639641  2.622002e-02 -8.613421 -8.699425  2.897548e-02   
252.200000  44    -8.643941  3.142145e-02 -8.612520 -8.703615  3.371043e-02   
275.975000  45    -8.649511  3.714635e-02 -8.612365 -8.709036  3.886961e-02   
299.750000  46    -8.656867  4.355673e-02 -8.613310 -8.716176  4.468577e-02   
323.525000  47    -8.666214  5.157747e-02 -8.614636 -8.725198  5.211375e-02   
347.300000  48    -8.677188  6.282499e-02 -8.614363 -8.735734  6.273568e-0

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')